In [1]:
import pandas as pd

In [ ]:
# set base url for API
base_url = 'https://api.beta.ons.gov.uk/v1/data?'

In [5]:
# set URIs
energy_uri = 'uri=energy/electricity-and-gas-consumption'


In [6]:
# combine to form a link
link = base_url + energy_uri

link

'https://api.beta.ons.gov.uk/v1/data?uri=energy/electricity-and-gas-consumption'